In [18]:
import numpy as np
import scipy.io as sio 
import scipy.signal as sig
from matplotlib import pyplot as plt
%matplotlib inline
dat = sio.loadmat('/Users/soumilhooda/Desktop/BCI/BCICIV_4_mat/sub1_comp.mat',struct_as_record = True)

In [19]:
train_data = dat['train_data']
train_dg = dat['train_dg']
samples,channels = train_data.shape
tt = 400 #Sampling time
sf = 1000 #Sampling frequency 
channel_data = np.transpose(train_data,(1,0))
channel_label = np.transpose(train_dg, (1,0))
samples_finger,fingers = train_dg.shape

In [26]:
print(samples_finger)

400000


In [20]:
# FIR filter 
def bandPass(signal,fl,fh):
    taps = 505
    band = np.array([fl,fh])
    h = sig.firwin(taps,band,fs=sf,pass_zero='bandpass')
    output = np.convolve(signal,h,mode='same')
    return output

In [21]:
# Amplitude modulation : 
def AM(signal):
    cur = 0
    output = []
    for i in range(samples):
        if(i and i % 40 == 0):
            output.append(cur)
            cur = 0
        cur += signal[i]**2
    output.append(cur)
         
    return output

In [22]:
# Amplitude modulation : 
def AM_finger(signal):
    cur = 0
    output = []
    for i in range(samples_finger):
        if(i and i % 40 == 0):
            output.append(cur)
            cur = 0
        cur += signal[i]**2
    output.append(cur)
         
    return output

In [23]:
# Seperate channel_data into frequency bands : 
sub_band = [bandPass(x,1,60) for x in channel_data]
gamma_band = [bandPass(x,60,100) for x in channel_data]
fast_gamma_band = [bandPass(x,100,200) for x in channel_data]

In [24]:
# Perform amplitude modulation on each band :
sub_band = [AM(x) for x in sub_band]
gamma_band = [AM(x) for x in gamma_band]
fast_gamma_band = [AM(x) for x in fast_gamma_band]

In [36]:
finger_downsampled = []
for i in range (len(train_dg)):
    if i % 40 == 0:
        finger_downsampled.append(train_dg[i])

In [37]:
print(len(finger_downsampled))

10000


In [38]:
np.savetxt('finger_flexion_downsampled_sub1',finger_downsampled)